In [ ]:
import time

import numpy as np

from scipy import special, integrate

from matplotlib import pyplot as plt

from tqdm.notebook import tqdm

import sympy

import regions

Might need to do some analytic calculations...

In [ ]:
oo = sympy.oo

In [ ]:
R, Ie, Re, n, bn, R1 = sympy.symbols('R, I_e, R_e, n, b_n, R_1', real=True, positive=True)

In [ ]:
sersic = Ie * sympy.exp(-bn*((R/Re)**(1/n)-1))
sersic

Start by plotting the sersic fomula up to scope out where the integral might be tractible:

In [ ]:
ICOEFFS = [2, -1/3, 4/405, 46/25515, 131/1148175, -2194697/30690717750]
def f_bn(n):
    """
    Ciotti and Bertin 99, valid for n >~ 0.36
    """
    return np.sum([C*n**(1-i) for i, C in enumerate(ICOEFFS)], axis=0)

def sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return Ie * np.exp(-bn*((R/Re)**(1/n)-1))

def log_sersic_profile(R, Ie, Re, n):
    bn = f_bn(n)
    return np.log(Ie) - bn*((R/Re)**(1/n)-1)

In [ ]:
r = np.linspace(0, 3, 1025)[1:]
for ni in [.5, 1, 2, 3, 4]:
    plt.plot(r, sersic_profile(r, 1, 1, ni), label=ni)
plt.legend(loc=0)
plt.axvline(1, c= 'k', ls=':')

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
r = np.linspace(0, 10, 1025)[1:]
for ax, ni in zip(axs.ravel(), [.5, 1, 2, 4, 6, 8]):
    ax.semilogx(r, sersic_profile(r, 1, 1, ni))
    ax.set_title(f'$n={ni}$')
    ax.axvline(1, c= 'k', ls=':')

Now lets see how the integrand behaves since what we really need is to integrate R*I dr:

In [ ]:
r = np.linspace(0, 5, 1025)[1:]
for ni in [.5, 1, 2, 3, 4]:
    plt.plot(r, r*sersic_profile(r, 1, 1, ni), label=ni)
plt.legend(loc=0)
plt.axvline(1, c= 'k', ls=':')

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))
r = np.logspace(-6, 3, 1024*4 + 1)[1:]
for ax, ni in zip(axs.ravel(), [.5, 1, 2, 4, 6, 8]):
    ax.semilogx(r, r*sersic_profile(r, 1, 1, ni))
    ax.set_title(f'$n={ni}$')
    ax.axvline(1, c= 'k', ls=':')

Ok, so the integrand is dominated by the peak and out to ~0.4-2 orders of magnitude above/below it

Can we analytically work out the peak by solving where the derivative is 0?

In [ ]:
solns = sympy.solve(sympy.diff(R*sersic, R), R)
assert len(solns)==1
solns[0]

Well that's shockingly straightfoward...

In [ ]:
def integral_peak(Re, n):
    bn = f_bn(n)
    return Re*(n/bn)**n

test_ns = np.array([.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
peaks = integral_peak(1, test_ns)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(test_ns, peaks, '-o')
ax2.semilogy(test_ns, peaks, '-o')

Analytical solutions for the circular integral?

In [ ]:
infintegral = sympy.integrate(R*sersic*2*sympy.pi, (R,0, oo))
infintegral

In [ ]:
defintegral = sympy.integrate(R*sersic*2*sympy.pi, (R,0, R1))
defintegral

Implement this integral as a optimized scipy/numpy-compatible numerical version:

In [ ]:
def sersic_integral(Router, Re, n):
    bnval = f_bn(n)
    numer1 = 4 * np.pi * Re * Re * bnval**(-2*n) * n*n * np.exp(bnval)
    
    # non-trivial thing here: the scipy gammainc is a *regularized* gamma function meaning it has a factor of 1/gamma(s), 
    # so we square our prefactor to have scipy's gammainc match the sympy definition which is not regularized
    numer2 = special.gamma(2*n)**2 * special.gammainc(2*n, Router**(1/n) * Re**(-1/n) * bnval)
    
    denom = special.gamma(2*n + 1)

    return numer1 * numer2 / denom

Confirm that it matches the sympy derivation for some test values, allowing for some rounding since sympy's engine isn't quite the same:

In [ ]:
Ro_val = 3.5
Re_val = 1.2
n_val = 1.45

scipy_val = sersic_integral(Ro_val, Re_val, n_val)
subs = {Re:Re_val, n:n_val, R1:Ro_val, bn: f_bn(n_val)}
sympy_val = (defintegral/Ie).evalf(subs=subs)

np.testing.assert_allclose(scipy_val, float(sympy_val))
scipy_val, sympy_val

## Integral tests

Now we need to try to work out a numerical solution of the integral for weird-shaped observing areas. We will try a few different approaches.  Set up some common bits for each:

In [ ]:
n_fid = 3.05
Re_fid = 1.02
Ie_fid = 1

infintegral.evalf(subs={n:n_fid, Ie:Ie_fid, Re: Re_fid, bn: f_bn(n_fid)}), defintegral.evalf(subs={n:n_fid, Ie:Ie_fid, Re: Re_fid, bn: f_bn(n_fid), R1:6})

In [ ]:
survey_region = regions.PolygonPixelRegion(regions.PixCoord(x=[-2, -2, 2, 2], y=[-2, 2, 10, -2]))

survey_region.plot()

x = np.linspace(-2, 2, 10)
def upper_line_y(x):
    return 2*x+6
plt.plot(x, upper_line_y(x), ':', c='r', lw=1)

plt.xlim(survey_region.bounding_box.ixmin-.1, survey_region.bounding_box.ixmax+.1)
plt.ylim(survey_region.bounding_box.iymin-.1, survey_region.bounding_box.iymax+.1)

In [ ]:
def make_r_grid(nr, nphi, outerr):
    phi = np.arange(nphi)*2*np.pi/nphi
    r = np.linspace(0, outerr, nr+1)[1:]
    
    return np.meshgrid(r, phi)

def make_r_grid_rweighted(nr, nphi, outerr):
    phi = np.arange(nphi)*2*np.pi/nphi
    u = np.linspace(0, 1, nr+1)[1:]
    r = np.sqrt(u)*outerr
    
    return np.meshgrid(r, phi)

# we use nphiouterhalf because that makes the total number reasonably close to the above 2
def make_r_grid_phiweighted(nr, nphiouterhalf, outerr):
    r = np.linspace(0, outerr, nr+1)[1:]
    rs = []
    phis = []
    for ri in r:
        nphi = int(ri/outerr * nphiouterhalf*2)
        phi = np.linspace(0, 2*np.pi, nphi+1)[:-1]
        rs.append(np.full_like(phi, ri))
        phis.append(phi)
    
    return np.concatenate(rs), np.concatenate(phis)

gridfuncs = [make_r_grid, make_r_grid_rweighted, make_r_grid_phiweighted]

fig, axs = plt.subplots(1, 3, subplot_kw=dict(projection='polar'), figsize=(12, 6))
for ax, f in zip(axs.ravel(), gridfuncs):
    fres = f(10, 30, 3)
    ax.scatter(*[c.ravel() for c in fres][::-1], s=10, alpha=.75)
    ax.set_title(f'n={len(fres[0].ravel())}')
None

In [ ]:
def grid_in_region(r, phi, region):
    x, y = r*np.cos(phi), r*np.sin(phi)
    msk = region.contains(regions.PixCoord(x=x, y=y))
    return r[msk], phi[msk], msk

### Technique 1: direct integration

This will be too slow but is an arbitrary-precision check to ensure accuracy of the others. Does not use the region mask above so a bit more awkward, but mocks up the region above.

In [ ]:
def sersic_integrand(x, y, Ie, Re, n):
    R = np.hypot(x, y)
    # don't need the R because dblquad is already over a  cartersian grid
    return sersic_profile(R, Ie, Re, n)

First do something large enough to be very close to infinity

In [ ]:
infresult = integrate.dblquad(sersic_integrand, 
                  a=-200, b=200, 
                  gfun=lambda x: -200, hfun=lambda x:200,#hfun=upper_line_y, 
                  args=(Ie_fid, Re_fid, n_fid))
infresult, infintegral.evalf(subs={n:n_fid, Ie:Ie_fid, Re: Re_fid, bn: f_bn(n_fid)})

Now try a more realistic case and compare to a vaguely similar definite integral

In [ ]:
directresult = integrate.dblquad(sersic_integrand, 
                  a=-2, b=2,
                  gfun=lambda x: -2, hfun=upper_line_y, 
                  args=(Ie_fid, Re_fid, n_fid))
directresult, defintegral.evalf(subs={n:n_fid, Ie:Ie_fid, Re: Re_fid, bn: f_bn(n_fid), R1:3})

### Technique 2: Direct grid evaluation

Compute the grid values and integrate numerically on the grid

In [ ]:
def sersic_integral_direct_grid(nr, nphi, Ie=Ie_fid, Re=Re_fid, n=n_fid, region=survey_region):
    outerr = np.max(region.bounding_box.extent)
    r, phi = make_r_grid(nr, nphi, outerr)
    dr = np.diff(r, axis=1).mean()
    dphi = np.diff(phi, axis=0).mean()
    rg, phig, msk = grid_in_region(r, phi, region)
    I = sersic_profile(rg, Ie, Re, n)

    return np.sum(I*rg*dr*dphi)

res = sersic_integral_direct_grid(100, 100)

# result, correct result, relative error
float(res), directresult[0], (float(res)-directresult[0])/directresult[0]

In [ ]:
def sersic_integral_direct_grid_rweighted(nr, nphi, Ie=Ie_fid, Re=Re_fid, n=n_fid, region=survey_region):
    outerr = np.max(region.bounding_box.extent)
    r, phi = make_r_grid_rweighted(nr, nphi, outerr)
    dr = np.insert(np.diff(r, axis=1).mean(axis=0), 0, r[0,0])
    #dr = np.insert(np.diff(r, axis=1).mean(axis=0), -1, r[-1,-1])
    dr = np.array([dr for _ in range(r.shape[0])])
    dphi = np.diff(phi, axis=0).mean()
    rg, phig, msk = grid_in_region(r, phi, survey_region)
    dr = dr[msk]
    I = sersic_profile(rg, Ie, Re, n)

    return np.sum(I*rg*dr*dphi)

res = sersic_integral_direct_grid_rweighted(100, 100)

# result, correct result, relative error
float(res), directresult[0], (float(res)-directresult[0])/directresult[0]

In [ ]:
def sersic_integral_direct_grid_phiweighted(nr, nphiouterhalf, Ie=Ie_fid, Re=Re_fid, n=n_fid, region=survey_region):
    outerr = np.max(region.bounding_box.extent)
    r, phi = make_r_grid_phiweighted(nr, nphiouterhalf, outerr)

    dr = np.diff(r)
    dr = dr[dr!=0].mean()

    dphi = np.diff(phi)
    dphi[dphi<0] = dphi[np.roll( dphi<0, -1)]
    dphi = np.insert(dphi, 0, dphi[0])

    rg, phig, msk = grid_in_region(r, phi, survey_region)
    dphi = dphi[msk]
    I = sersic_profile(rg, Ie, Re, n)

    return np.sum(I*rg*dr*dphi)

res = sersic_integral_direct_grid_phiweighted(100, 100)

# result, correct result, relative error
float(res), directresult[0], (float(res)-directresult[0])/directresult[0]

Now need to try over different grid sizes and see what converges fastest

In [ ]:
nrnphi_to_try = [(10,30),(30, 10), (10, 100), (100, 10), (100, 100), (500, 100), (100, 500), (1000, 1000)]
timing_repeats = 10

#### equal r/phi spacing

In [ ]:
times = []
medtimes = []
stdtimes = []
results = []
for nr, nphi in tqdm(nrnphi_to_try):
    reses = []
    ts = []
    for _ in range(timing_repeats):
        t1 = time.time()
        reses.append(sersic_integral_direct_grid(nr, nphi))
        t2 = time.time()
        ts.append(t2-t1)
    times.append(ts)
    medtimes.append(np.median(ts))
    stdtimes.append(np.std(ts))

    # confirm that there's no significant variation in the results at 1e-7 level
    assert np.std(reses)/np.mean(reses) < 1e-7, f'{nr},{nphi} {np.std(reses)} {np.mean(reses)} {np.array(reses)}'
    
    results.append(np.mean(reses))

fracdev = (np.array(results) - directresult[0])/directresult[0]

fig, (ax1, ax2) = plt.subplots(2, 1)

sc = ax1.scatter(*np.array(nrnphi_to_try).T, c=np.log10(medtimes), vmin=-4, vmax=0)
plt.colorbar(sc, ax=ax1).set_label('log10(median time)')

sc = ax2.scatter(*np.array(nrnphi_to_try).T, c=np.log10(np.abs(fracdev)), vmin=-5, vmax=0)
plt.colorbar(sc, ax=ax2).set_label('log10(fractional deviation')

for ax in [ax1, ax2]:
    ax.loglog()

for nl, t, s in zip(nrnphi_to_try, medtimes, stdtimes):
    print(nl, 't', f'{t:.2e} +/- {s:.2e}, frac: {s/t:.3%}')

for nl, f in zip(nrnphi_to_try, fracdev):
    print(nl, 'Ideviation:', f'{f:.3%}')
    

#### r weighted

In [ ]:
times = []
medtimes = []
stdtimes = []
results = []
for nr, nphi in tqdm(nrnphi_to_try):
    reses = []
    ts = []
    for _ in range(timing_repeats):
        t1 = time.time()
        reses.append(sersic_integral_direct_grid_rweighted(nr, nphi))
        t2 = time.time()
        ts.append(t2-t1)
    times.append(ts)
    medtimes.append(np.median(ts))
    stdtimes.append(np.std(ts))

    # confirm that there's no significant variation in the results at 1e-7 level
    assert np.std(reses)/np.mean(reses) < 1e-7, f'{nr},{nphi} {np.std(reses)} {np.mean(reses)} {np.array(reses)}'
    
    results.append(np.mean(reses))

fracdev = (np.array(results) - directresult[0])/directresult[0]

fig, (ax1, ax2) = plt.subplots(2, 1)

sc = ax1.scatter(*np.array(nrnphi_to_try).T, c=np.log10(medtimes), vmin=-4, vmax=0)
plt.colorbar(sc, ax=ax1).set_label('log10(median time)')

sc = ax2.scatter(*np.array(nrnphi_to_try).T, c=np.log10(np.abs(fracdev)), vmin=-5, vmax=0)
plt.colorbar(sc, ax=ax2).set_label('log10(fractional deviation')

for ax in [ax1, ax2]:
    ax.loglog()

for nl, t, s in zip(nrnphi_to_try, medtimes, stdtimes):
    print(nl, 't', f'{t:.2e} +/- {s:.2e}, frac: {s/t:.3%}')

for nl, f in zip(nrnphi_to_try, fracdev):
    print(nl, 'Ideviation:', f'{f:.2%}')
    

#### phi weighted

In [ ]:
times = []
medtimes = []
stdtimes = []
results = []
for nr, nphi in tqdm(nrnphi_to_try):
    reses = []
    ts = []
    for _ in range(timing_repeats):
        t1 = time.time()
        reses.append(sersic_integral_direct_grid_phiweighted(nr, nphi))
        t2 = time.time()
        ts.append(t2-t1)
    times.append(ts)
    medtimes.append(np.median(ts))
    stdtimes.append(np.std(ts))

    # confirm that there's no significant variation in the results at 1e-7 level
    assert np.std(reses)/np.mean(reses) < 1e-7, f'{nr},{nphi} {np.std(reses)} {np.mean(reses)} {np.array(reses)}'
    
    results.append(np.mean(reses))

fracdev = (np.array(results) - directresult[0])/directresult[0]

fig, (ax1, ax2) = plt.subplots(2, 1)

sc = ax1.scatter(*np.array(nrnphi_to_try).T, c=np.log10(medtimes), vmin=-4, vmax=0)
plt.colorbar(sc, ax=ax1).set_label('log10(median time)')

sc = ax2.scatter(*np.array(nrnphi_to_try).T, c=np.log10(np.abs(fracdev)), vmin=-5, vmax=0)
plt.colorbar(sc, ax=ax2).set_label('log10(fractional deviation')

for ax in [ax1, ax2]:
    ax.loglog()


for nl, t, s in zip(nrnphi_to_try, medtimes, stdtimes):
    print(nl, 't', f'{t:.2e} +/- {s:.2e}, frac: {s/t:.3%}')

for nl, f in zip(nrnphi_to_try, fracdev):
    print(nl, 'Ideviation:', f'{f:.2%}')
    

TODO: decide why the weighting methods are not right

### Technique 3: Integrate to outer edge and remove excess

Evaluate the integral to the largest point in the mask, then subtract out the parts of the radial grid that are outside

### Technique 4: Integrate to inner edge and add excess

Evaluate the integral to the smallest point in the mask, then add the parts of the radial grid that are outside the integral but inside the mask.

### Technique 5: Sample integrand points by inverse distribution, scale by outer edge

Start with technique 3, but then sample the integrand by CDF and just sum up the fraction in and out of the mask instead of evaluating the integral explicitly